In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup, Trainer , TrainingArguments
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from rouge_score import rouge_scorer
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_

In [4]:
tf.config.list_physical_devices('GPU')

[]

In [5]:
tf.config.list_physical_devices('GPU')

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [6]:
df = pd.read_csv("merge_df.csv")

In [7]:
df

,Unnamed: 0,File_path,Articles,Summaries
0,0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...,...
5444,5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [8]:
df = df.drop(columns="Unnamed: 0")

In [9]:
df

,File_path,Articles,Summaries
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...
5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [10]:
filtered_df = df[df['File_path'] == 'entertainment']

In [11]:
filtered_df

,File_path,Articles,Summaries
1328,entertainment,Super Size Me wins writers' award..Super Size ...,Spurlock was given his award on the same day t...
1329,entertainment,"Mogul Wilson backing UK rap band..Tony Wilson,...","Tony Wilson, the music mogul who established t..."
1330,entertainment,Police praise 'courageous' Ozzy..Rock star Ozz...,"""I could have been badly injured or shot or an..."
1331,entertainment,Eastwood's Baby scoops top Oscars..Clint Eastw...,The boxing drama was named best picture and Ea...
1332,entertainment,Actor Scott is new Bond favourite..Bookmaker W...,Bookmaker William Hill has stopped taking bets...
...,...,...,...
5380,entertainment,WASHINGTON — Senator Jeff Sessions was conf...,late tuesday republicans voted to formally sil...
5400,entertainment,"Was it just a year ago that Katie Holmes, Jenn...",aware that the payoff may be negligible many d...
5405,entertainment,Mother Jones was named magazine of the year on...,also honored were pacific standard for feature...
5408,entertainment,"Damien Chazelle, the writer and director of th...",what was it like to shoot there we were able t...


In [12]:
def remove_emojis_and_symbols(text):
    # Remove emojis
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    # Remove non-alphanumeric symbols
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [13]:
filtered_df['Articles'] = filtered_df['Articles'].apply(remove_emojis_and_symbols)

C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\2494307040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Articles'] = filtered_df['Articles'].apply(remove_emojis_and_symbols)


In [14]:
filtered_df['Summaries'] = filtered_df['Summaries'].apply(remove_emojis_and_symbols)

C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\969203157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Summaries'] = filtered_df['Summaries'].apply(remove_emojis_and_symbols)


In [15]:
filtered_df

,File_path,Articles,Summaries
1328,entertainment,Super Size Me wins writers awardSuper Size Me ...,Spurlock was given his award on the same day t...
1329,entertainment,Mogul Wilson backing UK rap bandTony Wilson th...,Tony Wilson the music mogul who established th...
1330,entertainment,Police praise courageous OzzyRock star Ozzy Os...,I could have been badly injured or shot or any...
1331,entertainment,Eastwoods Baby scoops top OscarsClint Eastwood...,The boxing drama was named best picture and Ea...
1332,entertainment,Actor Scott is new Bond favouriteBookmaker Wil...,Bookmaker William Hill has stopped taking bets...
...,...,...,...
5380,entertainment,WASHINGTON Senator Jeff Sessions was confi...,late tuesday republicans voted to formally sil...
5400,entertainment,Was it just a year ago that Katie Holmes Jenni...,aware that the payoff may be negligible many d...
5405,entertainment,Mother Jones was named magazine of the year on...,also honored were pacific standard for feature...
5408,entertainment,Damien Chazelle the writer and director of the...,what was it like to shoot there we were able t...


In [16]:
! pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=16fd3ff078074db0ab0d13fba77bc51b36ea21bfcbf92b5ea15e958d71851fdf
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


c:\Users\chunl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize and preprocess the text data
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
max_length = 1024  # Maximum sequence length

In [17]:
def tokenize_text(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True, padding='max_length', return_attention_mask=True)
    return inputs.to(device)  # Move the tokenized inputs to the GPU

def tokenize_summary(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=300, truncation=True, padding='max_length', return_attention_mask=True)
    return inputs.to(device)  # Move the tokenized summaries to the GPU


filtered_df['TokenizedText'] = filtered_df['Articles'].apply(tokenize_text)
filtered_df['TokenizedSummary'] = filtered_df['Summaries'].apply(tokenize_summary)

C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\827673237.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TokenizedText'] = filtered_df['Articles'].apply(tokenize_text)
C:\Users\chunl\AppData\Local\Temp\ipykernel_3084\827673237.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TokenizedSummary'] = filtered_df['Summaries'].apply(tokenize_summary)


In [18]:
train_df, test_df = train_test_split(filtered_df, test_size=0.2, random_state=42)

In [19]:
X_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedText']])
Y_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedSummary']])
X_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedText']])
Y_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedSummary']])

In [20]:
train_dataset = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [31]:
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['Articles']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summaries']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [33]:
from datasets import Dataset, DatasetDict

In [34]:
articles = DatasetDict()

In [36]:
data_train, data_test = train_test_split(df, test_size=0.10, random_state=42)

In [38]:
train_ds = Dataset.from_pandas(data_train, preserve_index=False)
test_ds = Dataset.from_pandas(data_test, preserve_index=False)

In [39]:
articles['train'] = train_ds
articles['test'] = test_ds

In [40]:
dataset_train = articles['train']
dataset_valid = articles['test']

In [42]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [44]:
from transformers import BartTokenizer

In [45]:
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/4904 [00:01<?, ? examples/s]


NameError: name 'tokenizer' is not defined

In [21]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [22]:
# Create a GradScaler for mixed-precision training
scaler = GradScaler()

In [23]:
# Define hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the GPU




BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [26]:
import evaluate

In [27]:
rouge = evaluate.load("rouge")

In [28]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [29]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [30]:
training_args = TrainingArguments(
    output_dir="results_bart",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="results_bart",
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0003,
    dataloader_num_workers=4
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=len(train_dataloader) * 10)  # Add learning rate scheduler
early_stopping_rounds = 2
best_rouge_score = -1
current_round = 0

In [78]:
# Define gradient accumulation steps
accumulation_steps = 20  # You can adjust this number

def train(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(tqdm(dataloader, desc="Training")):
        inputs = batch[0].to(device)  # Move the input batch to the GPU
        attention_mask = (inputs != 0).float().to(device)  # Create attention mask
        targets = batch[1].to(device)  # Move the target batch to the GPU

        with autocast():
            outputs = model(input_ids=inputs, attention_mask=attention_mask, decoder_input_ids=targets, labels=targets)
            loss = outputs.loss

        # Perform gradient accumulation
        loss = loss / accumulation_steps
        scaler.scale(loss).backward()

        if (step + 1) % accumulation_steps == 0:
            # Update gradients and optimizer once every accumulation_steps
            clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional gradient clipping
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(dataloader)



In [79]:
for epoch in range(3):
    train_loss = train(model, train_dataloader, optimizer, scheduler)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

Training: 100%|██████████| 185/185 [06:16<00:00,  2.04s/it]


Epoch 1, Train Loss: 0.7028


Training: 100%|██████████| 185/185 [06:28<00:00,  2.10s/it]


Epoch 2, Train Loss: 0.7044


Training: 100%|██████████| 185/185 [06:28<00:00,  2.10s/it]

Epoch 3, Train Loss: 0.7034


In [83]:
def evaluate1(model, dataloader):
    model.eval()

    test_articles = []
    actual_summaries = []
    predicted_summaries = []
    rouge1_precision_scores = []

    scorer = rouge_scorer.RougeScorer(['rouge1'])

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Test"):
            inputs = batch[0].to(device)
            attention_mask = (inputs != 0).float().to(device)
            targets = batch[1].to(device)
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=512, num_beams=30, length_penalty=3.0, early_stopping=True)

            for output, target, input_text in zip(outputs, targets, inputs):
                # Calculate ROUGE-1 precision for each sample
                output_text = tokenizer.decode(output, skip_special_tokens=True)
                target_text = tokenizer.decode(target, skip_special_tokens=True)
                rouge_scores = scorer.score(output_text, target_text)
                rouge1_precision_scores.append(rouge_scores['rouge1'].precision)

                # Append tokenized text, actual summaries, and predicted summaries
                test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
                actual_summaries.append(target_text)
                predicted_summaries.append(output_text)

    return test_articles, actual_summaries, predicted_summaries, rouge1_precision_scores

In [84]:
def evaluate2(model, dataloader):
    model.eval()

    test_articles = []
    actual_summaries = []
    predicted_summaries = []
    rouge2_precision_scores = []

    scorer = rouge_scorer.RougeScorer(['rouge2'])

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Test"):
            inputs = batch[0].to(device)
            attention_mask = (inputs != 0).float().to(device)
            targets = batch[1].to(device)
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=512, num_beams=30, length_penalty=3.0, early_stopping=True)

            for output, target, input_text in zip(outputs, targets, inputs):
                # Calculate ROUGE-2 precision for each sample
                output_text = tokenizer.decode(output, skip_special_tokens=True)
                target_text = tokenizer.decode(target, skip_special_tokens=True)
                rouge_scores = scorer.score(output_text, target_text)
                rouge2_precision_scores.append(rouge_scores['rouge2'].precision)

                # Append tokenized text, actual summaries, and predicted summaries
                test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
                actual_summaries.append(target_text)
                predicted_summaries.append(output_text)

    return test_articles, actual_summaries, predicted_summaries, rouge2_precision_scores

In [85]:
# Evaluate the model
test_articles, actual_summaries, predicted_summaries, rouge1_precision_scores = evaluate1(model, test_dataloader)

# Create a dictionary with the extracted data
data = {
    'Article': test_articles,
    'Actual Summary': actual_summaries,
    'Predicted Summary': predicted_summaries,
    'ROUGE-1 Precision': rouge1_precision_scores,
}

# Create a Pandas DataFrame from the dictionary
results_rouge1_df = pd.DataFrame(data)


# Display the DataFrame
pd.set_option('display.max_colwidth', None)
results_rouge1_df.head(5)

Evaluating Test:   0%|          | 0/47 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 360.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 18.01 GiB is allocated by PyTorch, and 4.24 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
results_rouge1_df.to_excel('results_rouge11.xlsx')

In [ ]:
# Evaluate the model
test_articles, actual_summaries, predicted_summaries, rouge2_precision_scores = evaluate2(model, test_dataloader)

# Create a dictionary with the extracted data
data = {
    'Article': test_articles,
    'Actual Summary': actual_summaries,
    'Predicted Summary': predicted_summaries,
    'ROUGE-2 Precision': rouge2_precision_scores,
}

# Create a Pandas DataFrame from the dictionary
results_rouge2_df = pd.DataFrame(data)

results_rouge2_df.head(5)

Evaluating Test: 100%|██████████| 12/12 [4:06:39<00:00, 1233.31s/it] 


Article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Belle named best Scottish bandBelle  Sebastian have been named the best Scottish band of all time after a three monthlong public pollThe group beat Travis and Idlewild into second and third place respectively Franz Ferdinand who recently picked up five Brit Award nominations ended up in 15th place while the Eurythmics wound up at a lowly 38 Other Scottish acts such as the Mull Historical Society who also featured in the top 50 performed at a party in Glasgow where the result was announced Scottishbased band Snow Patrol who finished 14th in the vote and have been nominated for a pair of Brit Awards were among the performers who covered wellknown Scottish po

In [ ]:
results_rouge2_df.to_excel('results_rouge22.xlsx')

In [ ]:

model.save_pretrained("pre_train_sum_batch_4")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [54]:
abc="Samantha Bee called out Mark Meadows, who was White House chief of staff under Donald Trump, after 2,319 of his text messages revealed how deep he and other key Republicans were involved in the plot to overturn the election and stop Joe Biden from becoming president. ??More than 2,000 text messages!?? Bee said on ??Full Frontal?? on Thursday night. ??This proves definitively that Mark Meadows is a gossipy little bitch.?? The messages included Fox News host Sean Hannity taking orders from the White House on how to help with voter turnout on Election Day, Sen. Mike Lee (R-Utah) pleading for talking points after the election and Rep. Marjorie Taylor Greene (R-Ga.) calling for ??Marshall?? law. ??Marshall??s Law, as everyone knows, is that no one should pay retail prices for quality yoga pants,?? Bee cracked. Bee also noted that Meadows still has more than 1,000 messages that he hasn??t turned over. ??Damn! Mark Meadows, you are messy,?? she said. ??Delete my number!??Bee urged the Jan. 6 committee to push forward with its investigation. ??If there??s one thing these texts prove it??s that it doesn??t matter to Republicans if they know for a fact that Joe Biden won the election,?? she said. ??What matters is these human bumper stickers are more than willing to use lies and a violent base to both take and keep power.?? And she warned that if they??re not held accountable, it??ll happen again:"

In [55]:
input_ids = tokenizer.encode(abc, return_tensors="pt")

In [56]:
summary_ids = model.generate(input_ids=input_ids, max_length=1024, num_beams=17, length_penalty=2.0, early_stopping=False)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [57]:
generated_summary

'Samantha Bee called out Mark Meadows, who was White House chief of staff under Donald Trump, after 2,319 of his text messages revealed how deep he and other key Republicans were involved in the plot to overturn the election and stop Joe Biden from becoming president.?\x80?More than 2,000 text messages!?\x80” Bee said on?\x83?Full Frontal?\x80& on Thursday night.?\u2000?This proves definitively that Mark Meadows is a gossipy little bitch.?\x80?, Bee said. The messages included Fox News host Sean Hannity taking orders from the White House on how to help with voter turnout on Election Day, Sen. Mike Lee (R-Utah) pleading for talking points after the election, Rep. Marjorie Taylor, a member of the House Judiciary Committee, and Rep. David Cicilline, R-N.Y., calling for an investigation into Meadows’ alleged role in the conspiracy, according to Bee’s transcript of the Jan. 6 committee hearing, as well as a letter from Rep. Mark Meadows to Rep. Bob Goodlatte, D-Va., urging him to “delete my

In [69]:
import string

In [66]:
def clean_text(text):
    # Remove punctuation and convert to lowercase
    translator = str.maketrans("", "", string.punctuation)
    cleaned_text = text.translate(translator).lower()
    return cleaned_text

In [67]:
def rouge_1_evaluation(reference, generated):
    reference = clean_text(reference)
    generated = clean_text(generated)

    reference_tokens = reference.split()
    generated_tokens = generated.split()

    # Calculate ROUGE-1 recall
    overlapping_tokens = set(reference_tokens) & set(generated_tokens)
    recall = len(overlapping_tokens) / len(reference_tokens) if len(reference_tokens) > 0 else 0.0

    return recall

In [70]:
rouge_1_recall = rouge_1_evaluation(abc, generated_summary)
print(f"ROUGE-1 Recall: {rouge_1_recall}")

ROUGE-1 Recall: 0.6059322033898306


In [71]:
def rouge_2_evaluation(reference, generated):
    reference = clean_text(reference)
    generated = clean_text(generated)

    reference_bigrams = list(zip(reference.split()[:-1], reference.split()[1:]))
    generated_bigrams = list(zip(generated.split()[:-1], generated.split()[1:]))

    # Calculate ROUGE-2 recall
    overlapping_bigrams = set(reference_bigrams) & set(generated_bigrams)
    recall = len(overlapping_bigrams) / len(reference_bigrams) if len(reference_bigrams) > 0 else 0.0

    return recall

In [73]:
rouge_2_recall = rouge_2_evaluation(abc, generated_summary)
print(f"ROUGE-2 Recall: {rouge_2_recall}")

ROUGE-2 Recall: 0.7489361702127659


In [4]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [30]:
model.from_pretrained("facebook/bart-base")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [9]:
abc = "Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloé Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing with bad reviews in film and TV. Melissa McCarthy said in an interview that she confronted a critic at the Toronto Film Festival who had previously criticised her for her appearance in 2014 film Tammy. Jennifer Lawrence said in Variety that she gets defensive after reading negative reviews of her work. You're so in the zone, you put your whole soul and body, you move to shoot a movie, and you then love it, obviously because you wouldn't be there if you didn't love it, and then people just destroy it, she explained."

In [11]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [31]:
input_ids = tokenizer.encode("summarize:"+ abc, return_tensors="pt")

In [32]:
summary_ids = model.generate(input_ids=input_ids, max_length=1024, num_beams=50, length_penalty=1.0, early_stopping=True)
generated_summary = tokenizer.decode(summary_ids[1], skip_special_tokens=True)

In [36]:
generated_summary = tokenizer.decode(summary_ids[-1], skip_special_tokens=True)

In [ ]:
def evaluate2(model, dataloader):
    model.eval()

    test_articles = []
    actual_summaries = []
    predicted_summaries = []
    rouge2_precision_scores = []

    scorer = rouge_scorer.RougeScorer(['rouge2'])

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Test"):
            inputs = batch[0].to(device)
            attention_mask = (inputs != 0).float().to(device)
            targets = batch[1].to(device)
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=512, num_beams=30, length_penalty=3.0, early_stopping=True)

            for output, target, input_text in zip(outputs, targets, inputs):
                # Calculate ROUGE-2 precision for each sample
                output_text = tokenizer.decode(output, skip_special_tokens=True)
                target_text = tokenizer.decode(target, skip_special_tokens=True)
                rouge_scores = scorer.score(output_text, target_text)
                rouge2_precision_scores.append(rouge_scores['rouge2'].precision)

                # Append tokenized text, actual summaries, and predicted summaries
                test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
                actual_summaries.append(target_text)
                predicted_summaries.append(output_text)

    return test_articles, actual_summaries, predicted_summaries, rouge2_precision_scores

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [72]:
abc

"Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloé Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing with bad reviews i

In [92]:
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)


In [94]:
from torch.utils.data import Dataset, DataLoader

In [106]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Define your article


# Initialize BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Tokenize the article
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)

# Generate predictions
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)
predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary)

Predicted Summary: Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloek Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing

In [108]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Tokenize the article
inputs = tokenizer(abc, return_tensors='pt', max_length=1024, truncation=True)

# Generate predictions
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)
predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary)

Predicted Summary: Superhero clichés: The 45-year-old actor and stand up comedian played an alien who came to Earth to protect his planet, becoming a Bollywood star to blend in. But despite high anticipation for the film, which was directed by Oscar winner Chloé Zhao, it received a relatively poor 47% rating on Rotten Tomatoes and achieved the lowest score of any Marvel Cinematic Universe film in audience survey Cinema Score. Empire said at the time of release: Chloek Zhao's entry into the superhero world is assured, ambitious and told on a dizzyingly cosmic scale - but even it can't escape the clichés of superhero storytelling. The Guardian's Peter Bradshaw wrote: There are some nice touches and an attractive new diversity worn lightly, but this is an underpowered and uncertain film. Nanjiani and his wife, Emily V Gordon, were Oscar nominees in 2018 for their romantic comedy The Big Sick, in which he starred alongside Zoe Kazan. He's one of many stars who have spoken out about dealing